In [16]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv('train.csv')

In [4]:
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [7]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(columns=['Survived']),
    df['Survived'],
    test_size=0.2,
    random_state=42
)

In [ ]:
df.isna().sum() 

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

Strategy 
- simple imputer on age and embarked
- onehotencoding on sex and embarked

In [67]:
# were getting several problems when we used the below syntax because it removes null values from embared and do onehotencoding at the same time
t1 = ColumnTransformer(
    transformers=[
        ('tnf1', SimpleImputer(strategy ='mean'), ['Age']), #defult is mean only
        ('tnf2', SimpleImputer(strategy ='most_frequent'), ['Embarked']),
         ],
    remainder='passthrough'
)
t2 = ColumnTransformer(
    transformers=[
        ('tnf2', OneHotEncoder(sparse_output=False, drop='first'), ['remainder__Sex', 'tnf2__Embarked'])
    ],
    remainder='passthrough'
)
t1.set_output(transform="pandas")
t2.set_output(transform="pandas")

ColumnTransformer(remainder='passthrough',
                  transformers=[('tnf2',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['remainder__Sex', 'tnf2__Embarked'])])

In [68]:
x_train_new= t1.fit_transform(x_train)
x_test_new= t1.transform(x_test)

x_train_new= t2.fit_transform(x_train_new)
x_test_new= t2.transform(x_test_new)


In [39]:
x_train.shape

(712, 7)

In [69]:
dtree = DecisionTreeClassifier()

In [70]:
x_train_new.head()

,tnf2__remainder__Sex_male,tnf2__tnf2__Embarked_Q,tnf2__tnf2__Embarked_S,remainder__tnf1__Age,remainder__remainder__Pclass,remainder__remainder__SibSp,remainder__remainder__Parch,remainder__remainder__Fare
331,1.0,0.0,1.0,45.5,1,0,0,28.5000
733,1.0,0.0,1.0,23.0,2,0,0,13.0000
382,1.0,0.0,1.0,32.0,3,0,0,7.9250
704,1.0,0.0,1.0,26.0,3,1,0,7.8542
813,0.0,0.0,1.0,6.0,3,4,2,31.2750


In [71]:
dtree.fit(x_train_new, y_train)

DecisionTreeClassifier()

In [72]:
y_pred = dtree.predict(x_test_new)

In [74]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7932960893854749

In [76]:
dtree.predict([[1.0,	0.0,	1.0,	23.0,	2,	0	,0	,13.0000]])

c:\Users\rajar\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([0], dtype=int64)